# Notebook Lecture 7: Feedback and the Root Locus Method
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook introduces the concept of feedback control and presents a first method for feedback control analysis with the Root Locus method.

Authors:
- Felix Muller; fmuller@ethz.ch
- Mark Benazet Castells; mbenazet@ethz.ch

## Learning Objectives
This lecture introduces feedback control and the Root Locus method, a classical control technique for analyzing closed-loop system behavior and stability.

After completing this material, you should be able to:

- Understand how feedback control modifies system behavior:
 - Stabilizing unstable systems
 - Improving responsiveness
 - Reducing oscillations

- Apply the Root Locus method to:
 - Sketch root locus plots using Evans' rules 
 - Determine closed-loop pole locations for varying gains
 - Analyze system stability and performance


### Import the packages:

The following cell imports the required packages. Run it before running the rest of the notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from control import tf, rlocus, poles, feedback, impulse_response
import ipywidgets as widgets
from ipywidgets import interact
import warnings

## Feedback Control System Behavior

Feedback control is a powerful technique that can fundamentally alter how a system behaves. By measuring the system's output and comparing it to a desired reference, a controller can adjust the system's input to achieve the desired output. This simple concept has far-reaching implications for system behavior, allowing us to stabilize unstable systems, improve responsiveness, and reduce oscillations.

### 1. Stabilizing Unstable Systems 

Recall the inverted pendulum with:
- Length: L = $\frac{3}{2}$
- Mass: m = $1$
- Moment of inertia: J = $\frac{1}{3}mL^2$ = $\frac{3}{4}$
- Damping coefficient: c = $\frac{9}{4}$
- Input torque: $\tau_f$

<div style="text-align:center;">
    <img src="./img/pendulum.png" alt="Inverted Pendulum" width="500">
</div>

The transfer function when linearized around the unstable equilibrium point is:

$$G(s) = \frac{3}{s^2 + 3s - 10}$$

#### Open Loop Characteristics

<div style="text-align:center;">
    <img src="./img/OL.png" alt="Inverted Pendulum" width="500">
</div>

The characteristic equation is:
$$s^2 + 3s - 10 = 0$$

The poles of the system are:
$$\lambda_1 = 2, \quad \lambda_2 = -5$$

As we see we have an unstable system, which makes sense are we are linearizing around the unstable upright position and therefore any small perturbation will make the system unstable.

#### Closed Loop Characteristics

Now let's add a feedback controller to stabilize the system. We will use a proportional controller with gain $K$:

$$G_c(s) = K$$

<div style="text-align:center;">
    <img src="./img/CL.png" alt="Inverted Pendulum" width="500">
</div>

The closed-loop transfer function is:

$$G_{cl}(s) = \frac{G(s)}{1 + G(s)G_c(s)} = \frac{3K}{s^2 + 3s - 10 + 3K}$$


The characteristic equation is:

$$s^2 + 3s - 10 + 3K = 0$$

The poles of the system are:

$$\lambda_{1,2} = \frac{-3 \pm \sqrt{3^2 + 4(10 - 3K)}}{2}$$

To achieve asymptotic stability, we need all eigenvalues to have strictly negative real part. Therefore, the system is stable if:

$$49 - 12K < 9 \Rightarrow K > \frac{10}{3}$$


In [ ]:
# Function to plot either open-loop or closed-loop impulse response based on mode
def plot_impulse_response_toggle(K, mode):
    # Convert string input to lists of floats
    num = [3]
    den = [1, 3, -10]
    
    # Create the open-loop transfer function
    sys = tf(num, den)
    
    if mode == 'Open-Loop':
        # Get the impulse response for the open-loop system
        t, y = impulse_response(sys)
        title = 'Impulse Response of the Open-Loop System'
        label = 'Open-Loop Impulse Response'
        color = 'green'
    else:
        # Closed-loop transfer function for negative feedback
        closed_loop_sys = feedback(K * sys, 1)
        # Get the impulse response for the closed-loop system
        t, y = impulse_response(closed_loop_sys)
        title = 'Impulse Response of the Closed-Loop System'
        label = f'Closed-Loop Impulse Response for K = {K}'
        color = 'blue'
    
    # Plot the impulse response
    plt.figure()
    plt.plot(t, y, label=label, color=color, lw=2)
    plt.title(title, fontsize=14)
    plt.xlabel('Time (s)', fontsize=12)
    plt.ylabel('Response', fontsize=12)
    plt.grid(True)
    plt.legend()
    plt.show()

# Interactive function for students to input transfer function and toggle between open-loop and closed-loop
def interact_with_impulse_response_toggle():
    
    # Slider to vary K for closed-loop system
    K_slider = widgets.FloatSlider(min=0, max=10, step=0.1, value=1, description='Gain (K):')
    
    # Toggle button to switch between open-loop and closed-loop modes
    mode_button = widgets.ToggleButtons(
        options=['Open-Loop', 'Closed-Loop'],
        description='System Mode:',
        disabled=False
    )
    
    # Combine input boxes, slider, and button in a vertical layout
    ui = widgets.VBox([K_slider, mode_button])
    
    # Connect the input and output with the plot function
    out = widgets.interactive_output(plot_impulse_response_toggle, {
        'K': K_slider,
        'mode': mode_button
    })
    
    # Display the UI and output
    display(ui, out)

# Call the function to toggle between open-loop and closed-loop impulse response
interact_with_impulse_response_toggle()


## Root Locus Method

The Root Locus method is a graphical technique in control theory to analyze and design control systems. 
It helps in determining how the poles of the closed-loop system vary as the gain (or another system parameter) is varied. This is important to determine if the system is stable or meets our design criteria.

We will now plot the Root Locus in two different ways.

In the first plot, you can enter an open loop transfer function and then change the gain k and see how that changes the poles and zeros.

In [ ]:
# Global variable to store the last transfer function
previous_num = ''
previous_den = ''
pole_trajectories = []

# Function to compute poles for a given K and open-loop transfer function
def plot_poles_with_K(K, num_str, den_str):
    global previous_num, previous_den, pole_trajectories

    # Check if the transfer function has changed
    if num_str != previous_num or den_str != previous_den:
        pole_trajectories = []  # Reset the pole trajectory
        previous_num = num_str  # Update the stored numerator
        previous_den = den_str  # Update the stored denominator

    # Convert string input to lists of floats
    num = [float(n) for n in num_str.split()]
    den = [float(d) for d in den_str.split()]
    
    # Create the open-loop transfer function
    sys = tf(num, den)
    
    # Closed-loop transfer function for negative feedback
    closed_loop_sys = feedback(K * sys, 1)
    
    # Get poles of the closed-loop system
    poles = closed_loop_sys.poles()
    
    # Append the current poles to the trajectory list
    pole_trajectories.append(poles)
    
    # Plot the poles
    plt.figure()
    plt.axhline(0, color='black', lw=1)
    plt.axvline(0, color='black', lw=1)
    
    # Plot all previous poles to form the "trajectory"
    for previous_poles in pole_trajectories:
        plt.scatter(np.real(previous_poles), np.imag(previous_poles), color='blue', alpha=0.5)
    
    # Plot the current poles in red
    plt.scatter(np.real(poles), np.imag(poles), color='red', zorder=5, label=f'K = {K}')
    
    plt.grid(True)
    plt.title(f'Poles for K = {K}')
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.xlim([-5, 5])
    plt.ylim([-5, 5])
    plt.legend()
    plt.show()

# Interactive function for students to input open-loop transfer function and vary K
def interact_with_poles():
    global pole_trajectories
    pole_trajectories = []  # Initialize the trajectory
    
    # Input boxes for open-loop transfer function numerator and denominator
    num_box = widgets.Text(value='1', description='Numerator:', placeholder='Enter numerator coefficients (space-separated)')
    den_box = widgets.Text(value='1 2 1', description='Denominator:', placeholder='Enter denominator coefficients (space-separated)')
    
    # Slider to vary K
    K_slider = widgets.FloatSlider(min=0, max=10, step=0.1, value=1, description='Gain (K):')
    
    # Combine input boxes and slider in a vertical layout
    ui = widgets.VBox([num_box, den_box, K_slider])
    
    # Connect the input and output with the plot function
    out = widgets.interactive_output(plot_poles_with_K, {
        'K': K_slider,
        'num_str': num_box,
        'den_str': den_box
    })
    
    # Display the UI and output
    display(ui, out)

# Call the interactive function
interact_with_poles()

If you do this for every k, you will have drawn the root locus. To show that, you can now enter the same transfer function and use the Root Locus function built-in in the Controls library. You will see that every point you plotted above lies on the Root Locus.

In [ ]:
# Function to plot the root locus for the given transfer function
def plot_root_locus(num_str, den_str):
    # Suppress warnings for this function
    warnings.filterwarnings("ignore")

    # Convert string input to lists of floats
    num = [float(n) for n in num_str.split()]
    den = [float(d) for d in den_str.split()]
    
    # Create the open-loop transfer function
    sys = tf(num, den)
    
    # Plot the root locus
    plt.figure()
    rlocus(sys, plot=True)
    plt.title('Root Locus Plot')
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.grid(False)
    plt.xlim([-5, 5])
    plt.ylim([-5, 5])
    plt.show()

# Interactive function for students to input transfer function and plot the root locus
def interact_with_root_locus():
    # Input boxes for open-loop transfer function numerator and denominator
    num_box = widgets.Text(value='1', description='Numerator:', placeholder='Enter numerator coefficients (space-separated)')
    den_box = widgets.Text(value='1 2 1', description='Denominator:', placeholder='Enter denominator coefficients (space-separated)')
    
    # Combine input boxes in a vertical layout
    ui = widgets.VBox([num_box, den_box])
    
    # Connect the input and output with the plot function
    out = widgets.interactive_output(plot_root_locus, {
        'num_str': num_box,
        'den_str': den_box
    })
    
    # Display the UI and output
    display(ui, out)

# Call the function to plot root locus
interact_with_root_locus()